# 1) Titolo e obiettivi

Lezione 27: Unsupervised Feature Engineering - Creare feature potenti senza etichette

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, perché feature engineering unsupervised | 5 min |
| 2 | Teoria profonda: 3 famiglie di feature | 15 min |
| 3 | Schema mentale: workflow creazione e selezione | 5 min |
| 4 | Demo: PCA features, cluster features, anomaly scores | 30 min |
| 5 | Esercizi guidati + pipeline completa | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Creare **feature da PCA** | Sai estrarre PC, loadings, reconstruction error? |
| 2 | Creare **feature da clustering** | Sai usare label, distanze, probabilità GMM? |
| 3 | Creare **feature da anomaly detection** | Sai usare IF/LOF scores come colonne? |
| 4 | **Selezionare feature** senza label | Sai applicare VarianceThreshold e filtro correlazione? |
| 5 | Costruire **pipeline end-to-end** | Sai concatenare tutto in un workflow riproducibile? |

---

## L'idea centrale: arricchire il dataset senza label

```
DATI ORIGINALI:                  DATI ARRICCHITI:

X1   X2   X3   X4   X5          X1  X2  X3  X4  X5  PC1  PC2  cluster  dist_centroid  anomaly_score
─────────────────────   +   ──────────────────────────────────────────────────────────────────────
10   20   30   40   50          10  20  30  40  50  2.1  -0.5    0         0.12           0.03
...                             ...

Nuove feature:                   Vecchie feature filtrate:
├── PC1, PC2 (compressione)     └── Rimosso X3 (correlato 0.98 con X2)
├── cluster_label               └── Rimosso X5 (varianza < 0.01)
├── dist_centroid
└── anomaly_score_IF
```

---

## Le 3 famiglie di feature unsupervised

| Famiglia | Feature prodotte | Quando utili |
|----------|------------------|--------------|
| **PCA** | Componenti (PC), loadings, reconstruction_error | Ridurre rumore, decorrelation |
| **Clustering** | label, dist_centroid, cluster_prob (GMM) | Segmentazione, segnali di gruppo |
| **Anomaly** | IF_score, LOF_score, anomaly_flag | Risk features, segnali di qualità |

---

## Perché è potente

| Vantaggio | Spiegazione |
|-----------|-------------|
| **Non serve target** | Puoi arricchire PRIMA di avere label |
| **Cattura struttura** | Cluster, densità, varianza sono informativi |
| **Migliora modelli downstream** | XGBoost con cluster_label performa meglio |
| **Riduce rumore** | VarianceThreshold e correlazione eliminano feature inutili |

---

## Schema workflow

```
┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│   Raw Data   │ →   │    Scale     │ →   │   Crea FE    │
└──────────────┘     └──────────────┘     └──────────────┘
                                                  │
                           ┌──────────────────────┼──────────────────────┐
                           │                      │                      │
                    ┌──────▼──────┐       ┌───────▼───────┐      ┌───────▼───────┐
                    │  PCA feat   │       │ Cluster feat  │      │ Anomaly feat  │
                    └──────┬──────┘       └───────┬───────┘      └───────┬───────┘
                           │                      │                      │
                           └──────────────────────┼──────────────────────┘
                                                  ▼
                                     ┌──────────────────────┐
                                     │   Concatena tutto    │
                                     └──────────────────────┘
                                                  │
                           ┌──────────────────────┼──────────────────────┐
                           │                      │                      │
                    ┌──────▼──────┐       ┌───────▼───────┐      ┌───────▼───────┐
                    │ Var filter  │       │ Corr filter   │      │  Final DF     │
                    └─────────────┘       └───────────────┘      └───────────────┘
```

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| PCA | Lezione 24 | Sai estrarre componenti e varianza? |
| K-Means, GMM | Lezioni 20, clustering | Sai usare fit_predict e transform? |
| IF, LOF | Lezione 26 | Sai estrarre decision_function? |
| Correlazione | Lezione base | Sai calcolare corr() e filtrarla? |

**Cosa useremo:** StandardScaler, PCA, KMeans, GaussianMixture, IsolationForest, LocalOutlierFactor, VarianceThreshold.

# 2) Teoria concettuale
## 2.1 Perche' fare feature engineering unsupervised
- Nuove feature migliorano separabilita' e stabilita' dei modelli, anche senza label.
- Decorrelare (PCA) e sintetizzare distanze/score rende piu' semplice la fase di clustering/anomaly detection successiva.
- Feature selection riduce rumore e tempi di calcolo, evitando overfitting su pattern casuali.


## 2.2 Tipi di feature da creare
- Da PCA: componenti principali, varianza spiegata, reconstruction error per rilevare punti mal ricostruiti.
- Da clustering: label di appartenenza, distanza dal centroide, probabilita' (con GMM) per catturare appartenenze soft.
- Da anomaly detection: score IF/LOF e flag binari come segnali di rischio.


## 2.3 Selezione e valutazione
- Variance threshold: elimina feature quasi costanti (input matrice scalata, output feature ridotte); errore tipico: applicarla prima dello scaling.
- Filtro di correlazione: rimuove colonne altamente correlate per ridurre ridondanza; attenzione a mantenere le piu' interpretabili.
- Valutazione: controlla distribuzioni, varianza spiegata e stabilita' dei cluster prima/ dopo le nuove feature.


# 3) Schema mentale / mappa decisionale
Workflow: carica -> scala -> crea nuove feature (PCA/cluster/score) -> concatena -> seleziona -> valida con metriche (varianza, correlazioni) -> usa nei modelli.
Decision map sintetica:
1. Scala le feature numeriche.
2. Se servono feature dense e compatte: PCA.
3. Se vuoi segnali di appartenenza: clustering per label/distanze.
4. Se vuoi segnali di rischio: anomaly scores.
5. Filtra feature a bassa varianza e ad alta correlazione.
6. Documenta le trasformazioni (necessario per riproducibilita').


# 4) Sezione dimostrativa
- Demo 1: Features da PCA (componenti, loadings, reconstruction error).
- Demo 2: Features da clustering (label e distanze ai centroidi).
- Demo 3: Features da anomaly detection (score e flag).
- Demo 4: Feature selection unsupervised (varianza e correlazione).
- Demo 5: Pipeline completa di feature engineering.


## Demo 1 - Features da PCA
Perche': ottenere feature decorrelate e misurare la varianza spiegata. Checkpoint: nessun NaN, varianza cumulata dichiarata, reconstruction error con media > 0 ma non enorme.


In [1]:
# Demo 1: features da PCA
# Scopo: creare componenti principali, analizzare loadings e calcolare reconstruction error come feature aggiuntiva.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification

np.random.seed(42)
plt.close('all')

# Dataset simulato
X, _ = make_classification(n_samples=500, n_features=6, n_informative=4, n_redundant=0, random_state=42)
print(f"Shape iniziale: {X.shape}")
assert not np.isnan(X).any(), "NaN nei dati"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=3, random_state=42)
X_pca = pca.fit_transform(X_scaled)
var_cum = pca.explained_variance_ratio_.sum()
print(f"Varianza spiegata cumulata: {var_cum:.3f}")
assert X_pca.shape == (500, 3), "Shape PCA inattesa"

# Loadings
loadings = pd.DataFrame(pca.components_.T, columns=['PC1','PC2','PC3'])
loadings['feature'] = [f'feat_{i}' for i in range(X.shape[1])]
print(loadings[['feature','PC1','PC2','PC3']].head())

# Reconstruction error
X_reconstructed = pca.inverse_transform(X_pca)
recon_error = np.mean((X_scaled - X_reconstructed)**2, axis=1)
print(f"Reconstruction error medio: {recon_error.mean():.4f}")

# Dataset arricchito
pc_df = pd.DataFrame(X_pca, columns=['pc1','pc2','pc3'])
pc_df['recon_error'] = recon_error
print(pc_df.head())


Shape iniziale: (500, 6)
Varianza spiegata cumulata: 0.623
  feature       PC1       PC2       PC3
0  feat_0 -0.268777  0.689764 -0.185658
1  feat_1  0.076210  0.347309  0.921509
2  feat_2  0.030249  0.531830 -0.296600
3  feat_3  0.557793  0.221670 -0.014882
4  feat_4  0.645974 -0.129437 -0.050501
Reconstruction error medio: 0.3769
        pc1       pc2       pc3  recon_error
0 -2.340224 -0.859562  0.832477     1.538057
1  0.829370  1.355790 -2.610694     0.510795
2  2.131989  3.079159  1.147079     0.282756
3 -0.109150  0.187645 -1.400600     0.210626
4  2.322739 -0.127321  1.362790     0.191162


## Demo 2 - Features da clustering
Perche': usare appartenenza e distanza per arricchire il dataset. Checkpoint: cluster presenti, distanze finite, label in range previsto.


In [2]:
# Demo 2: features da clustering
# Scopo: aggiungere label e distanze ai centroidi come feature.
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from scipy.spatial.distance import cdist

np.random.seed(42)

X_blobs, _ = make_blobs(n_samples=400, centers=4, n_features=4, cluster_std=1.0, random_state=42)
print(f"Shape blobs: {X_blobs.shape}")
assert not np.isnan(X_blobs).any(), "NaN nei dati"

scaler_blobs = StandardScaler()
X_blobs_scaled = scaler_blobs.fit_transform(X_blobs)

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_blobs_scaled)
centroids = kmeans.cluster_centers_

# Distanze ai centroidi
all_dists = cdist(X_blobs_scaled, centroids)
min_dist = all_dists.min(axis=1)

clust_df = pd.DataFrame(X_blobs_scaled, columns=[f'feat_{i}' for i in range(X_blobs.shape[1])])
clust_df['cluster_label'] = labels
clust_df['min_dist_centroid'] = min_dist

print(clust_df.head())
print(f"Cluster unici: {np.unique(labels)}")
assert clust_df['cluster_label'].nunique() == 4, "Numero cluster inatteso"


Shape blobs: (400, 4)
     feat_0    feat_1    feat_2    feat_3  cluster_label  min_dist_centroid
0  1.080266 -0.828261 -0.126870 -1.462834              2           0.264878
1  1.306175 -0.788549 -0.215939 -1.687295              2           0.175003
2 -0.486784  1.361946  1.996068 -0.157401              1           0.332965
3  0.997877 -0.702270 -0.363898 -1.660943              2           0.399115
4 -0.272549  1.217380  1.513722 -0.106040              1           0.271966
Cluster unici: [0 1 2 3]


## Demo 3 - Features da anomaly detection
Perche': aggiungere segnali di rischio tramite score IF/LOF. Checkpoint: almeno qualche anomalia individuata, score definiti per tutti i campioni.


In [3]:
# Demo 3: features da anomaly detection
# Scopo: aggiungere score IF/LOF e flag binario come feature.
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import precision_recall_fscore_support

iso = IsolationForest(contamination=0.05, random_state=42)
iso.fit(X_blobs_scaled)
iso_scores = -iso.decision_function(X_blobs_scaled)  # piu' alto = piu' anomalo
iso_flag = (iso.predict(X_blobs_scaled) == -1).astype(int)

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_preds = lof.fit_predict(X_blobs_scaled)
lof_scores = -lof.negative_outlier_factor_
lof_flag = (lof_preds == -1).astype(int)

anom_df = clust_df.copy()
anom_df['iso_score'] = iso_scores
anom_df['lof_score'] = lof_scores
anom_df['anomaly_flag'] = ((iso_flag + lof_flag) > 0).astype(int)

print(anom_df[['cluster_label','min_dist_centroid','iso_score','lof_score','anomaly_flag']].head())
print(f"Anomalie rilevate (union IF/LOF): {anom_df['anomaly_flag'].sum()}")
assert anom_df['anomaly_flag'].sum() > 0, "Nessuna anomalia rilevata"


   cluster_label  min_dist_centroid  iso_score  lof_score  anomaly_flag
0              2           0.264878  -0.081139   1.001623             0
1              2           0.175003  -0.094364   0.949224             0
2              1           0.332965  -0.057322   1.059820             0
3              2           0.399115  -0.027677   1.170746             0
4              1           0.271966  -0.085961   1.032514             0
Anomalie rilevate (union IF/LOF): 28


## Demo 4 - Feature selection unsupervised
Perche': rimuovere feature quasi costanti o ridondanti per snellire il dataset. Checkpoint: numero feature ridotto, nessun NaN dopo la selezione.


In [4]:
# Demo 4: feature selection unsupervised
# Scopo: rimuovere feature quasi costanti e altamente correlate.
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

np.random.seed(42)

n = 300
base = np.random.randn(n)
noisy = base + np.random.normal(0, 0.05, n)  # altamente correlata
low_var = np.ones(n) * 0.5 + np.random.normal(0, 1e-3, n)  # quasi costante
other = np.random.randn(n)

raw_df = pd.DataFrame({
    'base': base,
    'noisy': noisy,
    'low_var': low_var,
    'other': other
})
print(raw_df.head())

scaler_sel = StandardScaler()
X_sel = scaler_sel.fit_transform(raw_df)

vt = VarianceThreshold(threshold=0.01)
X_vt = vt.fit_transform(X_sel)
cols_vt = raw_df.columns[vt.get_support()]
print(f"Feature dopo variance threshold: {list(cols_vt)}")

# Filtro di correlazione
sel_df = pd.DataFrame(X_vt, columns=cols_vt)
corr = sel_df.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
final_df = sel_df.drop(columns=to_drop)
print(f"Feature rimosse per alta correlazione: {to_drop}")
print(f"Shape finale: {final_df.shape}")
assert final_df.shape[1] >= 1, "Troppe feature rimosse"


       base     noisy   low_var     other
0  0.496714  0.455264  0.500757  0.368673
1 -0.138264 -0.166273  0.499078 -0.393339
2  0.647689  0.685053  0.500870  0.028745
3  1.523030  1.553548  0.501356  1.278452
4 -0.234153 -0.235198  0.500413  0.191099
Feature dopo variance threshold: ['base', 'noisy', 'low_var', 'other']
Feature rimosse per alta correlazione: ['noisy']
Shape finale: (300, 3)


## Demo 5 - Pipeline completa
Perche': combinare PCA, clustering e anomaly scores in una sola funzione riutilizzabile. Checkpoint: tutte le feature generate senza NaN, documentazione dei passaggi.


In [5]:
# Demo 5: pipeline completa di feature engineering
# Scopo: combinare PCA, clustering e anomaly scores in un'unica funzione.
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

np.random.seed(42)

X_pipe, _ = make_blobs(n_samples=300, centers=5, n_features=5, cluster_std=1.2, random_state=42)

scaler_pipe = StandardScaler()
X_pipe_scaled = scaler_pipe.fit_transform(X_pipe)

pca_pipe = PCA(n_components=2, random_state=42)
X_pipe_pca = pca_pipe.fit_transform(X_pipe_scaled)

kmeans_pipe = KMeans(n_clusters=5, random_state=42, n_init=10)
labels_pipe = kmeans_pipe.fit_predict(X_pipe_scaled)

iso_pipe = IsolationForest(contamination=0.03, random_state=42)
iso_scores_pipe = -iso_pipe.fit(X_pipe_scaled).decision_function(X_pipe_scaled)

feat_df = pd.DataFrame(X_pipe_scaled, columns=[f'feat_{i}' for i in range(X_pipe.shape[1])])
feat_df['pc1'] = X_pipe_pca[:,0]
feat_df['pc2'] = X_pipe_pca[:,1]
feat_df['cluster_label'] = labels_pipe
feat_df['iso_score'] = iso_scores_pipe

print(feat_df.head())
assert not feat_df.isna().any().any(), "Feature generate con NaN"


     feat_0    feat_1    feat_2    feat_3    feat_4       pc1       pc2  \
0  0.468270  1.340971  0.577778  1.508467 -0.846079  0.912879  0.043047   
1 -0.617514 -0.952589  0.899108  1.451887  1.424678 -0.964266  2.274673   
2  0.900779  1.246943  0.519948  0.724840 -0.696687  0.759260 -0.422689   
3 -0.972408 -0.975656  0.659182  1.070862  2.384483 -1.309268  2.618381   
4 -0.590719  1.193593  0.840604 -1.351147 -0.652050  1.943754 -0.318552   

   cluster_label  iso_score  
0              0  -0.078389  
1              4  -0.077979  
2              0  -0.066929  
3              4   0.003107  
4              3  -0.077650  


# 5) Esercizi svolti (passo-passo)
## Esercizio 27.1 - Feature engineering per segmentazione clienti
Obiettivo: creare componenti PCA, label/distanze KMeans e anomaly score per un dataset clienti simulato.


In [6]:
# Esercizio 27.1: feature engineering segmentazione clienti
# Passi: dataset clienti simulato, PCA, clustering, distanze, anomaly score.
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from scipy.spatial.distance import cdist

np.random.seed(42)

n = 600
recency = np.random.exponential(30, n)
frequency = np.random.poisson(5, n)
monetary = np.random.gamma(2, 100, n)
tenure = np.random.uniform(3, 48, n)
returns = np.random.binomial(5, 0.1, n)

customers = pd.DataFrame({
    'recency': recency,
    'frequency': frequency,
    'monetary': monetary,
    'tenure': tenure,
    'returns': returns
})
print(customers.head())
assert not customers.isna().any().any(), "NaN nel dataset clienti"

scaler_cust = StandardScaler()
X_cust_scaled = scaler_cust.fit_transform(customers)

pca_cust = PCA(n_components=3, random_state=42)
X_cust_pca = pca_cust.fit_transform(X_cust_scaled)

kmeans_cust = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_cust = kmeans_cust.fit_predict(X_cust_scaled)
centroids_cust = kmeans_cust.cluster_centers_
dists_cust = cdist(X_cust_scaled, centroids_cust).min(axis=1)

iso_cust = IsolationForest(contamination=0.03, random_state=42)
iso_scores_cust = -iso_cust.fit(customers).decision_function(customers)

cust_feat = pd.DataFrame(X_cust_pca, columns=['pc1','pc2','pc3'])
cust_feat['cluster_label'] = labels_cust
cust_feat['dist_centroid'] = dists_cust
cust_feat['iso_score'] = iso_scores_cust

print(cust_feat.head())
print(f"Cluster trovati: {np.unique(labels_cust)}")
assert cust_feat.shape[0] == n, "Numero di righe incoerente"


     recency  frequency    monetary     tenure  returns
0  14.078043          3  162.631454  34.351004        0
1  90.303643          3  204.711293  35.639688        2
2  39.502371          5  104.136746  40.637036        0
3  27.388277          6  334.970444  46.849769        0
4   5.088746          5  375.864185  46.902086        0
        pc1       pc2       pc3  cluster_label  dist_centroid  iso_score
0 -1.096118 -0.233232  0.895017              1       1.083076  -0.149867
1  1.243700 -0.257985 -0.430436              0       2.726563  -0.005324
2 -0.176321  0.002998  1.331163              1       1.485975  -0.145230
3  0.602547  0.002029  1.813016              1       1.959760  -0.115807
4  0.097305 -0.389702  1.803214              1       2.080375  -0.116216
Cluster trovati: [0 1 2]


## Esercizio 27.2 - Feature selection su dataset ridondante
Obiettivo: applicare variance threshold e filtro di correlazione confrontando la dimensionalita' prima/dopo.


In [7]:
# Esercizio 27.2: feature selection
# Passi: dataset ridondante, variance threshold, filtro di correlazione.
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

np.random.seed(42)

n = 400
base = np.random.randn(n)
redundant = base + np.random.normal(0, 0.02, n)
low_var = np.ones(n) * 0.3 + np.random.normal(0, 1e-3, n)
indep = np.random.randn(n)

feat_df = pd.DataFrame({
    'base': base,
    'redundant': redundant,
    'low_var': low_var,
    'indep': indep
})
print(feat_df.head())

scaler_fs = StandardScaler()
X_fs = scaler_fs.fit_transform(feat_df)

vt = VarianceThreshold(threshold=0.01)
X_fs_vt = vt.fit_transform(X_fs)
cols_vt = feat_df.columns[vt.get_support()]
print(f"Dopo variance threshold: {list(cols_vt)}")

corr_df = pd.DataFrame(X_fs_vt, columns=cols_vt)
corr = corr_df.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.85)]
final_cols = [c for c in corr_df.columns if c not in to_drop]
final_df = corr_df[final_cols]
print(f"Feature rimosse per correlazione: {to_drop}")
print(f"Shape finale: {final_df.shape}")
assert final_df.shape[1] >= 1, "Tutte le feature rimosse"


       base  redundant   low_var     indep
0  0.496714   0.464826  0.300938  0.125225
1 -0.138264  -0.150252  0.299484 -0.429406
2  0.647689   0.647793  0.300096  0.122298
3  1.523030   1.523969  0.299538  0.543298
4 -0.234153  -0.243155  0.299566  0.048860
Dopo variance threshold: ['base', 'redundant', 'low_var', 'indep']
Feature rimosse per correlazione: ['redundant']
Shape finale: (400, 3)


## Esercizio 27.3 - Multi-grain feature engineering
Obiettivo: usare clustering a granularita' diverse (k=2,4,8) per creare feature che catturano pattern gerarchici.


In [8]:
# Esercizio 27.3: multi-grain feature engineering
# Passi: clustering a diverse granularita' per creare piu' feature di appartenenza/distanza.
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from scipy.spatial.distance import cdist

np.random.seed(42)

X_mg, _ = make_blobs(n_samples=400, centers=8, cluster_std=0.8, n_features=3, random_state=42)
scaler_mg = StandardScaler()
X_mg_scaled = scaler_mg.fit_transform(X_mg)

ks = [2, 4, 8]
features = {}
for k in ks:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_mg_scaled)
    dists = cdist(X_mg_scaled, km.cluster_centers_).min(axis=1)
    features[f'cluster_k{k}'] = labels
    features[f'dist_k{k}'] = dists

mg_df = pd.DataFrame(features)
print(mg_df.head())
assert mg_df.shape[0] == X_mg.shape[0], "Shape incoerente"


   cluster_k2   dist_k2  cluster_k4   dist_k4  cluster_k8   dist_k8
0           1  1.212824           3  0.892908           1  0.339002
1           1  0.866778           3  0.293014           6  0.057093
2           1  0.915563           3  0.294158           6  0.480133
3           0  1.285964           2  0.433740           7  0.149177
4           0  1.304341           2  0.336715           2  0.164887


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **PCA per decorrelazione** | PC sono combinazioni lineari ortogonali |
| 2 | **Clustering per segmentazione** | label e distanze catturano appartenenza |
| 3 | **Anomaly scores per rischio** | IF/LOF come feature numeriche continue |
| 4 | **VarianceThreshold per noise** | Elimina feature quasi costanti |
| 5 | **Filtro correlazione per ridondanza** | Rimuovi colonne duplicate di informazione |

---

## Confronto sintetico: quali feature creare

| Situazione | Feature consigliate | Perché |
|------------|---------------------|--------|
| Troppe feature correlate | PCA (PC1, PC2, ...) | Decorrelazione |
| Vuoi segnali di gruppo | cluster_label, dist_centroid | Segmenti interpretativi |
| Vuoi segnali di rischio | IF_score, LOF_score | Anomalie come numeriche |
| Feature quasi costanti | VarianceThreshold | Eliminazione automatica |
| Matrice dataset finale | Concatena + filtra | Pulizia finale |

---

## Reference card: metodi usati

| Metodo | Input | Output | Feature prodotte |
|--------|-------|--------|------------------|
| `PCA(n_components)` | X scalato | transform → PC | PC1, PC2, ..., recon_error |
| `KMeans(k)` | X scalato | predict → label | cluster_label |
| `km.transform(X)` | X scalato | distanze | dist_to_c0, dist_to_c1, ... |
| `GaussianMixture(k)` | X scalato | predict_proba | prob_c0, prob_c1, ... |
| `IsolationForest` | X scalato | decision_function | if_score |
| `LocalOutlierFactor` | X scalato | negative_outlier_factor_ | lof_score |
| `VarianceThreshold(threshold)` | X | X filtrato | meno colonne |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| Feature NaN dopo concat | Indici non allineati | Usa .reset_index(drop=True) |
| Distanze tutte simili | k troppo alto o basso | Prova diversi k |
| Anomaly score tutti 0 | contamination = 0 | Usa valore realistico (0.01-0.05) |
| Troppe colonne rimosse | Soglia varianza/correlazione troppo alta | Abbassa soglia |
| Non scalare prima | Feature engineering su scale diverse | StandardScaler sempre |

---

## Template completo Feature Engineering Unsupervised

```python
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import VarianceThreshold

# 1) Carica e scala
X = ...  # DataFrame o array
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2) Feature da PCA
pca = PCA(n_components=5, random_state=42)
X_pca = pca.fit_transform(X_scaled)
pca_cols = [f'PC{i+1}' for i in range(5)]
df_pca = pd.DataFrame(X_pca, columns=pca_cols)
# Reconstruction error
X_recon = pca.inverse_transform(X_pca)
df_pca['recon_error'] = np.mean((X_scaled - X_recon)**2, axis=1)

# 3) Feature da clustering
km = KMeans(n_clusters=4, random_state=42, n_init='auto')
df_cluster = pd.DataFrame({
    'cluster_label': km.fit_predict(X_scaled)
})
dists = km.transform(X_scaled)
for i in range(dists.shape[1]):
    df_cluster[f'dist_c{i}'] = dists[:, i]

# 4) Feature da GMM (soft probabilities)
gmm = GaussianMixture(n_components=4, random_state=42)
probs = gmm.fit(X_scaled).predict_proba(X_scaled)
df_gmm = pd.DataFrame(probs, columns=[f'prob_c{i}' for i in range(4)])

# 5) Feature da anomaly detection
iso = IsolationForest(contamination=0.05, random_state=42)
iso.fit(X_scaled)
df_anomaly = pd.DataFrame({
    'if_score': iso.decision_function(X_scaled)
})
lof = LocalOutlierFactor(n_neighbors=30, contamination=0.05)
lof.fit_predict(X_scaled)
df_anomaly['lof_score'] = lof.negative_outlier_factor_

# 6) Concatena tutto
X_df = pd.DataFrame(X_scaled, columns=[f'X{i}' for i in range(X_scaled.shape[1])])
df_full = pd.concat([X_df, df_pca, df_cluster, df_gmm, df_anomaly], axis=1)

# 7) Selezione: VarianceThreshold
selector = VarianceThreshold(threshold=0.01)
X_var = selector.fit_transform(df_full)
print(f"Feature dopo VarianceThreshold: {X_var.shape[1]} / {df_full.shape[1]}")

# 8) Selezione: rimuovi correlate > 0.95
corr_matrix = df_full.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
df_final = df_full.drop(columns=to_drop)
print(f"Feature dopo filtro correlazione: {df_final.shape[1]} / {df_full.shape[1]}")
```

---

## Prossimi passi

| Lezione | Argomento | Collegamento |
|---------|-----------|--------------|
| 28 | Progetto End-to-End Unsupervised | Applica tutto su dataset reale |
| 29+ | AI, NLP, Deep Learning | Estensioni avanzate |

# 7) Checklist di fine lezione
- [ ] Ho scalato le feature prima di PCA, clustering e anomaly detection.
- [ ] Ho calcolato varianza spiegata e reconstruction error per le componenti PCA.
- [ ] Ho verificato label e distanze dei cluster e che non ci siano NaN.
- [ ] Ho aggiunto almeno uno score di anomalia come feature.
- [ ] Ho applicato variance threshold e filtro di correlazione se presenti feature ridondanti.
- [ ] Ho documentato le trasformazioni per riapplicarle in produzione.

Glossario (termini usati):
- Componenti principali (PC): combinazioni lineari delle feature originali che massimizzano la varianza.
- Loadings: pesi delle feature sulle componenti.
- Reconstruction error: differenza tra dati originali e ricostruiti dopo PCA.
- Cluster label: appartenenza a un centroide.
- Distanza dal centroide: metrica di vicinanza al proprio cluster.
- Probabilita' di appartenenza: output soft di GMM.
- Isolation score / LOF score: misure di anomalia (alto/basso = piu' anomalo a seconda del segno).
- Variance threshold: rimozione feature con varianza minima.
- Filtro di correlazione: rimozione di feature fortemente correlate.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-22 | Creazione: PCA e clustering features |
| 1.1 | 2024-02-01 | Aggiunto anomaly scores come feature |
| 2.0 | 2024-02-10 | Integrata pipeline completa con selezione |
| 2.1 | 2024-02-15 | Refactor con controlli e checkpoint |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII workflow creazione/selezione, 3 famiglie feature (PCA/cluster/anomaly), 5 take-home messages, template completo con GMM probabilità, reference card metodi, filtro correlazione automatico, errori comuni |

---

## Note per lo studente

Questa lezione chiude il toolkit **feature engineering senza label**:

| Tecnica | Cosa produce | Quando usarla |
|---------|--------------|---------------|
| PCA | PC, recon_error | Decorrelazione, compressione |
| Clustering | label, distanze, prob | Segmentazione, appartenenza |
| Anomaly | scores | Segnali di rischio |
| Selezione | meno colonne | Pulizia, velocità |

**Workflow tipico:**
1. Scale → 2. Crea feature → 3. Concatena → 4. Filtra → 5. Usa nei modelli

**Prossima tappa:** Lesson 28 - Progetto End-to-End che mette insieme tutto.